In [1]:
pip install mediapipe opencv-python

You should consider upgrading via the 'c:\users\kusha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
import sys
from threading import Timer

# Initializing mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

# Initializing mediapipe pose class.
mp_pose = mp.solutions.pose


In [3]:
# Function calculate_angle
'''
Inputs- 3 coordinates
Returns- Calculated angle
'''
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [4]:
# timeout function which increments counter after 8 seconds
def timeout():
    print("8 secs over!!")
    global counter
    counter +=1
    print(counter)


In [5]:
# Timer which calls function timeout after 8 seconds
t = Timer(8, timeout)

In [6]:
def display_text():
    cv2.rectangle(image, (250,0), (600,30), (245,117,16), -1)
    cv2.putText(image, 'Keep your knee bent', (250,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


In [7]:
cap = cv2.VideoCapture('C:/Kushal/Knee Bend Assignment 2/KneeBend.mp4')

# Knee bend counter variables
counter = 0 
stage = None
f=0
images_array=[]

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.95, min_tracking_confidence=0.95,model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if ret== False:
            break
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            leftHip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            leftKnee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            leftAnkle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(leftHip, leftKnee, leftAnkle)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(leftHip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            global t
            # If the leg is bent
            if angle < 160 and stage =='down':
                stage="up"
                if(t.is_alive()==False):
                        t = Timer(8, timeout)
                        t.start() 
                        print("Timer started")
                
            # if the leg is straight
            if angle > 160:
                stage = "down"
                if(t.is_alive()):
                    t.cancel()
                    display_text()
                    f=10
                    print("Timer cancelled")
  
            # for displaying "Keep your knee bent" for 10 frames so that it is clearly visible
            if(f):
                display_text()
                f-=1
        except:
            pass
        
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (15,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (90,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (85,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        
        images_array.append(image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Timer started
Timer cancelled
Timer started
8 secs over!!
1
Timer started
Timer cancelled
Timer started
Timer cancelled
Timer started
Timer cancelled
Timer started


In [8]:
len(images_array)

436

In [9]:
img= images_array[0]
height, width, layers = img.shape
size = (width,height)

In [10]:
# Combining images to form a video
out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
 
for i in range(len(images_array)):
    out.write(images_array[i])
out.release()

8 secs over!!
2
